<a href="https://colab.research.google.com/github/isabellacadisco/Algorithms-for-Massive-Data-course/blob/main/LSH7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
#set up os env
os.environ['KAGGLE_USERNAME'] = "isabellacadisco"
os.environ['KAGGLE_KEY'] = "847325691c2399e41dc9c0b78fbe499e"

# download dataset from kaggle and save it in drive
!kaggle datasets download -d xhlulu/medal-emnlp

100% 6.82G/6.82G [01:14<00:00, 94.9MB/s]
100% 6.82G/6.82G [01:14<00:00, 98.0MB/s]


In [2]:
!unzip medal-emnlp.zip #unzip dataset

Archive:  medal-emnlp.zip
  inflating: full_data.csv           
  inflating: pretrain_subset/test.csv  
  inflating: pretrain_subset/train.csv  
  inflating: pretrain_subset/valid.csv  


In [3]:
# set up spark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null #install java as VM
!wget -q https://dlcdn.apache.org/spark/spark-3.4.0/spark-3.4.0-bin-hadoop3.tgz #download Spark
!tar xf spark-3.4.0-bin-hadoop3.tgz #unzip the file
!pip install -q findspark

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.0-bin-hadoop3"

import findspark
findspark.init("spark-3.4.0-bin-hadoop3")# SPARK_HOME
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate() #SPARK SESSION

import pyspark
type(spark)

pyspark.sql.session.SparkSession

In [4]:
file_path = './full_data.csv'
data_df = spark.read.csv(file_path, header=True)

In [16]:
from pyspark.sql.functions import monotonically_increasing_id

In [17]:
dataset = data_df.select('TEXT').withColumn('doc_id', monotonically_increasing_id())

In [18]:
small_dataset = dataset.sample(withReplacement=False, fraction=0.000007, seed=42)

# Text processing

In [5]:
# TEXT PROCESSOR
!pip install spacy
!python -m spacy download en_core_web_sm

# Import spaCy and download the English model
import spacy
import en_core_web_sm

# Load the spaCy English model FORSE NON SERVE BOH
nlp = en_core_web_sm.load()

import string

2023-09-14 12:03:37.280174: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-14 12:03:39.143490: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 84.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [9]:
def preprocess_text(text):
    # Initialize spaCy resources
    nlp = spacy.load("en_core_web_sm")

    # Apply spaCy NLP pipeline
    doc = nlp(text)

    # Lemmatize and remove stopwords
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and token.text not in string.punctuation]

    # Join tokens back into a text
    cleaned_text = " ".join(tokens)

    return cleaned_text

In [10]:
s = preprocess_text("clinically")
print(s)

# .lemma_ returns lemma string

clinically


In [19]:
# UDF --

from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# Register the UDF with Spark
preprocess_udf_spark = udf(preprocess_text, StringType())

In [20]:
df_prep = small_dataset.withColumn("preprocessed_text", preprocess_udf_spark(small_dataset['TEXT']))

In [21]:
df_prep.show()

+--------------------+------------+--------------------+
|                TEXT|      doc_id|   preprocessed_text|
+--------------------+------------+--------------------+
|a recent outbreak...|      106539|recent outbreak s...|
|wefast gradientec...| 25769866227|wefast gradientec...|
|this paper deals ...| 42949764250|paper deal patien...|
|there is a close ...| 51539638497|close association...|
|occasional and un...| 51539642085|occasional unpred...|
|the relative perc...| 77309421895|relative percent ...|
|mixed alloys for ...|103079248410|mix alloy dental ...|
|a high incidence ...|103079293994|high incidence go...|
|a housetohouse su...|103079331152|housetohouse surv...|
|to characterize t...|120259124985|characterize cont...|
|we studied the ef...|120259129263|study efficacy ca...|
|the effects of cl...|137439020479|effect clonidine ...|
|lipopolysaccharid...|154618911990|lipopolysaccharid...|
|this case report ...|154618921437|case report yearo...|
|a study of one of...|163208873

In [22]:
df = df_prep.select('doc_id','preprocessed_text')

In [23]:
from pyspark.sql.functions import length

In [24]:
# Add a new column containing the character count for each row
df_with_char_count = df.withColumn("CharacterCount", length(df["preprocessed_text"]))

# Show the resulting DataFrame
df_with_char_count.show()

+------------+--------------------+--------------+
|      doc_id|   preprocessed_text|CharacterCount|
+------------+--------------------+--------------+
|      106539|recent outbreak s...|           518|
| 25769866227|wefast gradientec...|           617|
| 42949764250|paper deal patien...|           264|
| 51539638497|close association...|           251|
| 51539642085|occasional unpred...|           323|
| 77309421895|relative percent ...|           237|
|103079248410|mix alloy dental ...|           338|
|103079293994|high incidence go...|           498|
|103079331152|housetohouse surv...|           533|
|120259124985|characterize cont...|          1147|
|120259129263|study efficacy ca...|           775|
|137439020479|effect clonidine ...|           489|
|154618911990|lipopolysaccharid...|          1515|
|154618921437|case report yearo...|           601|
|163208873442|study af influenc...|            98|
|188978671675|rat fetuse day ge...|           682|
|206158468968|catenin polypepti

In [28]:
df_with_char_count.first()

Row(doc_id=106539, preprocessed_text='recent outbreak sh mouse laboratory animal centre surrey england describe strain mouse show clinical ps typical liver lesion nude nunu obese mouse inbreed strain show clinical symptom strain seroconverte neutralization typing mouse hepatitis virus show type hepatotropic strain low virulence finding consistent pathology disease recur outbreak disease cause strain strain mhv united kingdom rapid method diagnosis tissue culture devise employ peroxidase enzymelabele antibody technique pcd mouse hepatitis virus l cell', CharacterCount=518)

In [30]:
len("recent outbreak sh mouse laboratory animal centre surrey england describe strain mouse show clinical ps typical liver lesion nude nunu obese mouse inbreed strain show clinical symptom strain seroconverte neutralization typing mouse hepatitis virus show type hepatotropic strain low virulence finding consistent pathology disease recur outbreak disease cause strain strain mhv united kingdom rapid method diagnosis tissue culture devise employ peroxidase enzymelabele antibody technique pcd mouse hepatitis virus l cell")

518

In [25]:
from pyspark.sql.functions import avg

In [26]:
average_length = df_with_char_count.select(avg("CharacterCount")).collect()[0][0]


In [27]:
print(average_length)

673.2021276595744
